In [3]:
import pandas as pd
import os

In [2]:
df = pd.read_csv('pokemon.csv')

In [4]:
image_folder = 'images'

def get_image_path(pokemon_id):
    path = os.path.join(image_folder, f'{pokemon_id}.png')
    return path if os.path.exists(path) else None

In [5]:
df['image_path'] = df['Name'].apply(get_image_path)

In [8]:
# Check for missing images
missing = df[df['image_path'].isna()]
if not missing.empty:
    print(f"Warning: {len(missing)} images not found")
    print(missing['Name'].tolist())
    

In [9]:
print(df.head())

         Name  Type1   Type2   Evolution             image_path
0   bulbasaur  Grass  Poison     ivysaur   images\bulbasaur.png
1     ivysaur  Grass  Poison    venusaur     images\ivysaur.png
2    venusaur  Grass  Poison         NaN    images\venusaur.png
3  charmander   Fire     NaN  charmeleon  images\charmander.png
4  charmeleon   Fire     NaN   charizard  images\charmeleon.png
